<a href="https://colab.research.google.com/github/mwauquier/CUSO_dsm_tutorial/blob/main/CUSO_sem_distrib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SECTION 1 - Manipulation d'espaces vectoriels

Ce tutoriel propose d'utiliser gensim, une implémentation Python de Word2Vec, pour manipuler des modèles distributionnels. Pour plus d'informations sur gensim, et pour aller plus loin dans l'utilisation de cette librairie, visitez la [documentation en ligne](https://radimrehurek.com/gensim/models/word2vec.html), ou les [tutoriels dédiés](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html).  

On commence par charger la librairie `gensim`, c'est-à-dire l'ensemble des ressources, codes et fonctions nous permettant de manipuler les espaces vectoriels. Cela se fait à l'aide de la commande `import`

In [ ]:
# On charge la librairie gensim

import gensim.models.keyedvectors as word2vec

Il nous faut maintenant charger l'espace vectoriel (ou modèle distributionnel) sur lequel on souhaite travailler. Nous travaillerons ici avec des modèles directement mobilisables en ligne, pour des raisons pratiques - les modèles sont souvent des fichiers assez lourds, dont le transfert et le téléchargement peuvent être longs.

Pour cela, on va utiliser le module `downloader` de gensim qui nous permet d'accéder à des modèles disponibles en ligne.

In [ ]:
# On charge le module 'downloader' pour accéder aux modèles en ligne

import gensim.downloader

# La commande ci-dessous permet d'afficher le nom des modèles disponibles et que l'on peut charger

print(list(gensim.downloader.info()['models'].keys()))

Il nous suffit alors de choisir le modèle de notre choix  - on travaillera ici avec un modèle Word2vec, donc commençant par la mention *'word2vec'*. Les deux seuls corpus à ce titre étant donc 'word2vec-ruscorpora-300' et 'word2vec-google-news-300', nous choisirons ce dernier pour des raisons de lisibilité.

Il nous suffit donc de charger le modèle à l'aide de la fonction `gensim.downloader.load()`

In [ ]:
# On charge le modèle de notre choix dans la variable w2v_vectors
# Il nous suffira alors de faire appel à cette variable pour interroger le modèle

# Attention, le chargement peut être long !

w2v_vectors = gensim.downloader.load('word2vec-google-news-300')

L'espace vectoriel est désormais sauvegardé dans la variable *w2v_vectors*. Nous pouvons donc désormais l'interroger.

L'utilisation primaire d'un espace vectoriel est d'interroger la similarité distributionnelle (et donc par extension sémantique) de deux mots. Pour cela, on utilise la fonction `similarity()`.

In [ ]:
# On utilise la fonction 'similarity()' pour évaluer la proximité distributionnelle entre deux mots

w2v_vectors.similarity('WORD1','WORD2')

On peut plus largement identifier les *n* voisins distributionnels d'un mot cible, c'est-à-dire les *n* mots ayant les vecteurs les plus similaires à celui de notre mot cible. Pour cela, on utilise la fonction `most_similar()`.

In [ ]:
# On utilise la fonction 'most_similar()' pour trouver les voisins distributionnels d'un mot

w2v_vectors.most_similar('WORD', topn=10)

Dans l'absolu, on peut récupérer le vecteur d'un mot, c'est-à-dire ses coordonnées. Cela est notamment utile si l'on veut le réinjecter dans une autre chaîne de traitement (classification, clustering...). Mais cet objet est en lui-même difficile à interpréter.

In [ ]:
# On récupère le vecteur lui-même (ses coordonnées) de la façon suivante

vec = w2v_vectors['WORD']

# Mais comme on peut le voir, le vecteur est difficilement interprétable par lui-même
print(vec)

Parmi les fonctions comprises dans la librairie `gensim` pour explorer l'espace vectoriel, on retrouve par exemple la fonction `doesnt_match`, qui permet d'identifier dans une liste de mots un intrus sur la base de ses propriétés distributionnelles.

In [ ]:
# Parmi les petites fonctionnalités complémentaires, on peut identifier l'intrus dans une liste à l'aide de la fonction 'doesnt_match()'

print(w2v_vectors.doesnt_match(['WORD1', 'WORD2', 'WORD3', 'WORD4', 'WORD5', 'WORD6']))

On peut aussi complexifier un peu les fonctions de base, telle que `most_similar`, pour réaliser des tâches plus complexes. C'est notamment le cas de l'analogie (qu'est ce qui est à C ce que A est à B ?), que l'on peut représenter mathématiquement comme une équation de type `A - B + C = D`, où A, B, C et D sont des vecteurs. La fonction nous rapporte alors les mots qui répondent le mieux à ce qui est attendu de D.

In [ ]:
# On peut aussi compléter une analogie en reprenant la fonction 'most_similar()'
# L'idée est ici de répondre à la question : "quel est l'élément D qui est à l'élément C ce que l'élément A est à l'élément B ?"
# Qu'on peut traduire par A - B + C = D
# où A et C seront donc les deux mots que l'on mettra en "positive", B le mot en "negative", et D la réponse que nous retournera la fonction

w2v_vectors.most_similar(negative=['WORD1'],positive=['WORD2', 'WORD3'])

Vous pouvez ci-dessous essayer de faire vos propres requêtes et notamment : 
* Quels sont les voisins de *stock* ?
* Quels sont les voisins de *schtroumpf* ?
* Quelle est la proximité distributionnelle des paires suivantes ?
  * *good* et *bad*
  * *dog* et *beagle*
  * *game* et *play*
* Quels sont les voisins de *mouse* ?

Quelles conclusions tirez-vous de ces quelques exemples ?

# SECTION 2 - Entraînement de modèles

Entraîner (ou construire) un modèle distributionnel correspond au calcul des coordonnées des vecteurs à partir d'un corpus. L'entraînement d'un modèle n'est pas complexe.

Attention, le processus peut être plus ou moins long en fonction de votre machine (ou de celle du serveur sur lequel vous travaillez). Nous travaillerons ici avec de (tout) petits corpus, donc cela ne devrait pas prendre plus de quelques minutes.

Le système prend en entrée des phrases (des listes de mots) dans un fichier texte (.txt), et retourne un modèle (une liste de vecteurs). Dans les faits, nous allons lui fournir un fichier contenant une phrase par ligne, et où chaque mot est tokenisé.

Nous utiliserons pour commencer les corpus disponibles en ligne dans le Project Gutenberg. Pour cela, nous commençons par importer les librairies nécessaires.

In [ ]:
# Importation des librairires
# Ces librairies ne concernent que l'importation et le traitement des corpus du Project Gutenberg, et ne dépendent donc absolument pas de gensim
import nltk
from nltk.corpus import gutenberg

nltk.download('gutenberg')
nltk.download('punkt')

Cela va nous permettre de télécharger des corpus disponibles en ligne, dont voici la liste (*a priori*) exhaustive : 'austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt',
'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt',
'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt',
'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt',
'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt',
'shakespeare-macbeth.txt', 'whitman-leaves.txt'

Choisissez dans cette liste le corpus de votre choix, et notez bien son nom complet.

Ces corpus sont cependant au format brut (sans aucun traitement). Vous pouvez le voir en imprimant les premières lignes du texte.

In [ ]:
text = gutenberg.raw('XXX.txt')
print(text[:400]) # Nous imprimons ici les 400 premiers caractères du texte brut sauvegardé dans la variable "text"

Le système ne prend cependant pas un texte brut (non traité), mais des listes de mots. Nous devons donc appliquer un minimum de traitement à notre corpus.

La librairie `nltk` propose directement une fonction permettant de traiter ses corpus, `gutenberg.sents()`

In [ ]:
corpus = gutenberg.sents('XX.txt')

Il ne reste plus qu'à entraîner le modèle à l'aide de la fonction `gensim.models.Word2Vec()`

In [ ]:
model_default = gensim.models.Word2Vec(corpus)

Votre modèle est ici enregistré dans la variable *model_default*. Vous pouvez ainsi directement la mobiliser

In [ ]:
model_default.wv.similarity("WORD1","WORD2")

Notez ici l'utilisation du préfixe wv pour appeler la fonction. Cela est notamment dû aux différentes fonctions d'entraînement de Word2Vec avec gensim

Notez que cela ne s'utilise que si vous travaillez sur un modèle que vous venez tout juste d'entraîner dans votre instance, pas avec un modèle que vous importez (comme on le faisait précédemment).

Notez par ailleurs, comme son nom l'indique, que vous avez ici entraîné un modèle par défaut. Nous n'avons renseigné aucun paramètre autre que le corpus. Les hyper-paramètres ont donc des valeurs par défaut : dimensions des vecteurs = 100, fenêtre de 5, seuil de fréquence minimale de 5, CBOW.

Si on veut entraîner un modèle sur notre corpus avec un seuil de fréquence de 50, des vecteurs à 200 dimensions, une fenêtre de 3, et l'utilisation de l'architecture SkipGram, on utilisera la commande :

In [ ]:
model_advanced = gensim.models.Word2Vec(corpus, min_count=50, vector_size=200, window=3, sg=1)

Mais vous pouvez évidemment choisir les valeurs que vous souhaitez.

Vous pouvez aussi, si vous le souhaitez, entraîner un modèle à partir du petit corpus (extrait du BNC) présent dans le Drive associé à ce notebook. Pour cela, il faudra d'abord donner l'accord au notebook d'explorer le contenu du Google Drive. 

Notez que cette autorisation ne vaut que pour cette instance (elle est à renouveler à chaque utilisation du notebook), et le présent code n'amène à la modification d'aucun document.

In [ ]:
# Code valable si vous travaillez en ligne sur le notebook Colab

from google.colab import drive
drive.mount('/content/drive')

Il suffit alors d'utiliser la fonction `LineSentence()` de `gensim` (similaire à `guntenberg.sents()`) sur le corpus que vous indiquerez.

In [ ]:
from gensim.models.word2vec import LineSentence

corpus_bnc = LineSentence('/content/drive/MyDrive/CUSO_2022_semantique_distributionnelle/tiny_sample_bnc.txt')

Vous pouvez alors entraîner le modèle, comme précédemment, avec ou sans paramètres spécifiques.

In [ ]:
model_default_bnc = gensim.models.Word2Vec(corpus_bnc)

Vous pouvez dès lors comparer les représentations construites sur les différents corpus et/ou avec les différents paramètres. Par exemple, quels sont dans vos différents modèles les voisins de *man* ? Et de *stock* ?

Pour le moment, le modèle n'est enregistré que dans votre variable. Si vous fermez le programme, votre modèle disparaît. Vous n'aurez par ailleurs jamais strictement le même modèle.

Vous pouvez sauvegarder et exporter votre modèle avec la fonction `save()`

In [ ]:
word_vectors = model_default.wv
word_vectors.save("word2vec_default.wordvectors")

Le modèle peut alors être rechargé dans un programme avec la fonction `load()` vue précédemment.

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('word2vec_default.wordvectors)

# SECTION 3 - fastText

Vous pouvez tester un modèle fastText en chargeant celui via `gensim.downloader()` : `fasttext-wiki-news-subwords-300`.

In [ ]:
fastText_model = gensim.downloader.load('fasttext-wiki-news-subwords-300')

Ce modèle s'utilise de la même façon que les modèles word2vec.

In [ ]:
fastText_model.similarity('WORD1', 'WORD2')

In [ ]:
fastText_model.most_similar('WORD')